In [8]:
from utils.data_util import gather_by_annotator
from utils.data_util import gather_by_scene

from utils.data_util import read_mturk_annotation_multiple_scene
from utils.data_util import read_turkle_annotation_multiple_scene
from utils.data_util import read_annotation

from utils.data_util import generate_clusters_no_plural_combine_speakers, generate_all_clusters_combine_speakers, collect_mentions

In [9]:
import csv
import json
from copy import deepcopy
csv.field_size_limit(1131072)

1131072

In [10]:
mturk_1 = read_mturk_annotation_multiple_scene('data/pilot_3/mturk.csv')

mturk_2 = read_mturk_annotation_multiple_scene('data/pilot_3/single_annotator.csv')

golden_1 = read_turkle_annotation_multiple_scene('data/pilot_3/golden.csv')
for item in golden_1:
    item['WorkerId'] = "golden"

results = []
# results.extend(mturk_1)
results.extend(mturk_2)
results.extend(golden_1)

scenes = gather_by_scene(results)

In [35]:
mturk_1 = read_mturk_annotation_multiple_scene('data/pilot_3/mturk.csv')

mturk_2 = read_mturk_annotation_multiple_scene('data/pilot_3/single_annotator.csv')

golden_1 = read_turkle_annotation_multiple_scene('data/pilot_1/golden.csv')
for item in golden_1:
    item['WorkerId'] = "golden"

results = []
results.extend(mturk_1)
results.extend(mturk_2)
results.extend(golden_1)

scenes = gather_by_scene(results)

In [9]:
mturk_1 = read_mturk_annotation_multiple_scene('data/pilot_2/data_1.csv')

mturk_2 = read_mturk_annotation_multiple_scene('data/pilot_2/data_2.csv')

mturk_3 = read_mturk_annotation_multiple_scene('data/pilot_1/pilot_1.csv')

mturk_4 = read_mturk_annotation_multiple_scene('data/pilot_1/pilot_2.csv')

golden_1 = read_turkle_annotation_multiple_scene('data/pilot_1/golden.csv')
for item in golden_1:
    item['WorkerId'] = "golden_1"

golden_2 = read_turkle_annotation_multiple_scene('data/pilot_2/golden.csv')
for item in golden_2:
    item['WorkerId'] = "golden_2"

results = []
results.extend(mturk_1)
results.extend(mturk_2)
results.extend(mturk_3)
results.extend(mturk_4)
results.extend(golden_1)
results.extend(golden_2)

scenes = gather_by_scene(results)
#
# print(mturk_1[0]['WorkerId'])
# print(mturk_2[0]['WorkerId'])
# print(mturk_2[4]['WorkerId'])
# print(mturk_2[7]['WorkerId'])

In [21]:
mturk_1 = read_mturk_annotation_multiple_scene('data/mturk_bulk/bulk_1.csv')

mturk_2 = read_mturk_annotation_multiple_scene('data/mturk_bulk/bulk_2.csv')

mturk_3 = read_mturk_annotation_multiple_scene('data/mturk_bulk/bulk_3.csv')

results = []
results.extend(mturk_1)
results.extend(mturk_2)
results.extend(mturk_3)

In [23]:
print(len(results), 20+20+34)

193 74


In [36]:
def read_mturk_annotation_multiple_scene(path):
    """
    Load the annotation along with the document
    Output: sentence along with all annotations

    In this version, the input contains annotations from multiple scenes
    Different scenes could be split with sentence_offsets and query_offsets
    """
    output = []
    with open(path, 'r') as f:
        annotation_reader = csv.DictReader(f)
        for instance in annotation_reader:
            inputs = json.loads(instance["Input.json_data"])
            all_sentences = inputs['sentences']
            all_query_spans = inputs['querySpans']
            sentence_offsets = inputs['sentence_offsets']
            query_spans_offsets = inputs['querySpans_offsets']
            all_answer_spans = json.loads(instance["Answer.answer_spans"])

            for i in range(len(sentence_offsets) - 1):
                sent_start, sent_end = sentence_offsets[i], sentence_offsets[i + 1]
                query_start, query_end = query_spans_offsets[i], query_spans_offsets[i + 1]
                original_sentences = all_sentences[sent_start: sent_end]
                sentences = []
                for item in original_sentences:
                    temp_sent = []
                    if isinstance(item[0], list):
                        temp_sent.extend(item[0])
                        temp_sent.extend(item[1:])
                        sentences.append(temp_sent)
                    else:
                        sentences.append(item)

                query_spans = all_query_spans[query_start: query_end]
                for item in query_spans:
                    item['sentenceIndex'] -= sentence_offsets[i]

                answer_spans = all_answer_spans[query_start: query_end]
                for item in answer_spans:
                    item['querySpan']['sentenceIndex'] -= sentence_offsets[i]
                    # Process span list
                    temp_span_list = []
                    for x in item['span_list']:
                        if ('startToken' not in x) or ('endToken' not in x) or ('sentenceIndex' not in x):
                            continue
                        if x['startToken'] == -1 and x['endToken'] == -1:
                            continue
                        x['sentenceIndex'] -= sentence_offsets[i]
                        temp_span_list.append(x)
                    item['span_list'] = temp_span_list

                temp = {
                    "sentences": sentences,
                    "query_spans": query_spans,
                    "answer_spans": answer_spans,
                    "WorkerId": instance['WorkerId']
                }

                # Collect mentions into clusters
                temp['clusters_all'] = generate_all_clusters_combine_speakers(temp)
                temp['clusters_no_plural'] = generate_clusters_no_plural_combine_speakers(temp)

                # Collect mentions (For Kappa Cohen)
                answers = collect_mentions(temp)
                temp["answers"] = answers

                # Add to output
                output.append(temp)

    return output

In [23]:
mturk_1 = read_mturk_annotation_multiple_scene('data/mturk_bulk/bulk_2.csv')
results = []
results.extend(mturk_1)
annotator_results = gather_by_annotator(results)

for annotator in annotator_results:
    print(annotator, len(annotator_results[annotator]))

A2RBF3IIJP15IH 6
A3L2XKXABNO0N5 9
AKQAI78JTXXC9 21
ATR6RB1RULOC0 7


In [44]:
mturk_1 = read_mturk_annotation_multiple_scene('data/pilot_3/mturk.csv')
mturk_2 = read_mturk_annotation_multiple_scene('data/pilot_3/single_annotator.csv')
golden_1 = read_turkle_annotation_multiple_scene('data/pilot_3/golden.csv')
for item in golden_1:
    item['WorkerId'] = "golden_1"
results = []
# results.extend(mturk_1)
results.extend(mturk_2)
results.extend(golden_1)

annotator_results = gather_by_annotator(results)

for annotator in annotator_results:
    print(annotator, len(annotator_results[annotator]))

A1EVYY9TPBYTYB 3
golden_1 3


In [45]:
print(len(scenes))

3


In [12]:
keys = ['sentenceIndex', 'startToken', 'endToken']

for scene_key in scenes:
    print(scene_key)
    scene = scenes[scene_key]
    sentences = scene[0]['sentences']
    for sent in sentences:
        print(sent)
    spans = scene[0]['answer_spans']
    # Query
    for i in range(len(spans)):
        current_span = spans[i]
        query = [current_span['querySpan'][x] for x in keys]
        query_text = sentences[query[0]][query[1]: query[2]]
        # print(current_span)
        # if query_text[0]!="that":
        #     continue
        print(query, query_text)

        for k in range(len(scene)):
            annotator = scene[k]['WorkerId']
            current_span = scene[k]['answer_spans'][i]
            answers = []
            if (not current_span['notPresent']) and (not current_span['notMention']):
                for item in current_span['span_list']:
                    temp = [item[x] for x in keys]
                    temp.append(current_span['notSure'])
                    answers.append(temp)

            answer_texts = []
            for item in answers:
                answer_texts.append(sentences[item[0]][item[1]: item[2]])
            if not answers:
                if current_span['notPresent']:
                    print("notPresent", [x[-1] for x in answers], annotator)
                elif current_span['notMention']:
                    print("notMention", [x[-1] for x in answers], annotator)
            else:
                print(answers, answer_texts, [x[-1] for x in answers], annotator)
        print()

:|Where|'d|you|go|?|I|ca|n't
['Penny', ':', 'Where', "'d", 'you', 'go', '?', 'I', 'ca', "n't", 'tell', 'if', 'the', 'turkey', "'s", 'done', '!']
['Leonard', ':', 'Be', 'right', 'there', '!', 'Hi', ',', 'lover', '.']
['Penny', ':', 'What', 'are', 'you', 'doing', '?']
['Leonard', ':', 'I', "'", 'm', 'sorry', 'about', 'the', 'journal', '.', 'I', 'want', 'to', 'make', 'it', 'up', 'to', 'you', '.', 'So', 'I', "'", 'm', 'gon', 'na', 'let', 'you', 'post', 'a', 'shame', 'photo', 'of', 'me', 'on', 'Facebook', '.']
['Penny', ':', 'I', 'am', 'not', 'putting', 'that', 'on', 'the', 'Internet', '!', 'I', 'do', "n't", 'want', 'people', 'to', 'see', 'this', '.', 'I', 'do', "n't", 'want', 'to', 'see', 'it', '!']
['Leonard', ':', 'Do', "n't", 'want', 'people', 'to', 'see', 'what', ',', 'huh', '?', 'A', 'little', 'bit', 'of', 'this', '?']
['Penny', ':', 'Oh', '.']
['Leonard', ':', 'Some', 'of', 'this', '?', 'And', ',', 'since', 'it', "'s", 'Thanksgiving', ',', 'an', 'extra', 'helping', 'of', 'this', '?']

In [12]:
def print_annotations(scene):
    keys = ['sentenceIndex', 'startToken', 'endToken']
    sentences = scene['sentences']
    for sent in sentences:
        print(sent)
    spans = scene['answer_spans']
    # Query
    for i in range(len(spans)):
        current_span = spans[i]
        query = [current_span['querySpan'][x] for x in keys]
        query_text = sentences[query[0]][query[1]: query[2]]

        annotator = scene['WorkerId']
        current_span = scene['answer_spans'][i]
        answers = []
        if (not current_span['notPresent']) and (not current_span['notMention']):
            for item in current_span['span_list']:
                temp = [item[x] for x in keys]
                temp.append(current_span['notSure'])
                answers.append(temp)

        answer_texts = []
        for item in answers:
            answer_texts.append(sentences[item[0]][item[1]: item[2]])
        if answers==[]:
            if current_span['notPresent']:
                print(query, query_text, "notPresent", [x[-1] for x in answers])
            elif current_span['notMention']:
                print(query, query_text, "notMention", [x[-1] for x in answers])
        else:
            print(query, query_text, answers, answer_texts, [x[-1] for x in answers])
    print()

In [13]:
print(annotator_results["ATR6RB1RULOC0"][4])

KeyError: 'ATR6RB1RULOC0'

In [40]:

print_annotations(annotator_results["ATR6RB1RULOC0"][4])

['Missy', ':', 'Okay', '.', 'I', "'m", 'not', 'even', 'gon', 'na', 'ask', 'why', 'you', "'re", 'pimping', 'me', 'out', 'for', 'cheese', 'but', 'since', 'when', 'do', 'you', 'care', 'at', 'all', 'about', 'who', 'I', 'sleep', 'with', '?']
['Sheldon', ':', 'Truthfully', ',', 'I', "'ve", 'never', 'given', 'it', 'any', 'thought', 'but', 'it', 'has', 'been', 'pointed', 'out', 'to', 'me', 'that', 'you', 'carry', 'DNA', 'of', 'great', 'potential', '.']
['Missy', ':', 'What', 'on', 'earth', 'are', 'you', 'talking', 'about', '?']
['Sheldon', ':', 'Let', 'me', 'explain', '.', 'You', 'see', ',', 'I', "'m", 'a', 'superior', 'genetic', 'mutation', 'an', 'improvement', 'on', 'the', 'existing', 'mediocre', 'stock', '.']
['Missy', ':', 'And', 'what', 'do', 'you', 'mean', ',', '"', 'mediocre', 'stock', '"', '?']
['Sheldon', ':', 'That', 'would', 'be', 'you', '.', 'But', 'residing', 'within', 'you', 'is', 'the', 'potential', 'for', 'another', 'me', '.', 'Perhaps', 'even', 'taller', ',', 'smarter', 'and',

In [16]:
keys = ['sentenceIndex', 'startToken', 'endToken']

for scene in results:
    print(scene.keys())
    sentences = scene['sentences']
    spans = scene['answer_spans']
    annotator = scene['WorkerId']
    print(annotator)
    for j, sent in enumerate(sentences):
        print(sent)
        # Query
        for i in range(len(spans)):
            current_span = spans[i]
            query = [current_span['querySpan'][x] for x in keys]
            query_text = sentences[query[0]][query[1]: query[2]]
            if query[0]!=j:
                continue
            annotator = scene['WorkerId']
            current_span = scene['answer_spans'][i]
            answers = []
            if (not current_span['notPresent']) and (not current_span['notMention']):
                for item in current_span['span_list']:
                    temp = [item[x] for x in keys]
                    temp.append(current_span['notSure'])
                    answers.append(temp)

            answer_texts = []
            for item in answers:
                answer_texts.append(sentences[item[0]][item[1]: item[2]])
            if answers==[]:
                if current_span['notPresent']:
                    pass
                    print(query, query_text, "notPresent", [x[-1] for x in answers])
                elif current_span['notMention']:
                    print(query, query_text, "notMention", [x[-1] for x in answers])
            else:
                # print(query, query_text, answers, answer_texts, [x[-1] for x in answers])
                pass
            print()
        print('--')



    # for sent in sentences:
    #     print(sent)
    # print()
    # spans = scene['answer_spans']
    # # Query
    # for i in range(len(spans)):
    #     current_span = spans[i]
    #     query = [current_span['querySpan'][x] for x in keys]
    #     query_text = sentences[query[0]][query[1]: query[2]]
    #     print(query, query_text)
    #
    #     annotator = scene['WorkerId']
    #     current_span = scene['answer_spans'][i]
    #     answers = []
    #     if (not current_span['notPresent']) and (not current_span['notMention']):
    #         for item in current_span['span_list']:
    #             temp = [item[x] for x in keys]
    #             temp.append(current_span['notSure'])
    #             answers.append(temp)
    #
    #     answer_texts = []
    #     for item in answers:
    #         answer_texts.append(sentences[item[0]][item[1]: item[2]])
    #     if answers==[]:
    #         if current_span['notPresent']:
    #             print(annotator, "notPresent", [x[-1] for x in answers])
    #         elif current_span['notMention']:
    #             print(annotator, "notMention", [x[-1] for x in answers])
    #     else:
    #         print(annotator, answers, answer_texts, [x[-1] for x in answers])
    print("=="*50)

dict_keys(['sentences', 'query_spans', 'answer_spans', 'WorkerId', 'clusters_all', 'clusters_no_plural', 'answers'])
A2RBF3IIJP15IH
['Howard', ':', 'Watch', 'this', ',', 'it', "'s", 'really', 'cool', '.']
[0, 3, 4] ['this'] notMention []

[0, 5, 6] ['it'] notMention []

--
['Howard ’s phone', ':', 'Did', 'you', 'say', ':', '"', 'Call', 'Hellen', 'Boxlightner', '?']


--
['Howard', ':', 'No', '.', 'Call', 'Leonard', 'Hofstadter', '.']
[2, 2, 3] ['No'] notMention []


--
['Howard ’s phone', ':', 'Did', 'you', 'say', ':', '"', 'Call', 'Temple', 'Badsaidor', '?', '"']


--
['Howard', ':', 'No', '.']
--
['Leonard', ':', 'Here', ',', 'let', 'me', 'try', '.', 'Call', 'McFlono', 'McFloonyloo', '.', 'Heh', '-', 'heh', '.']


--
['Howard ’s phone', ':', 'Calling', 'Rajesh', 'Koothrappali', '.', '.']

--
['Raj', ':', 'Oh', ',', 'that', '’s', 'very', 'impressive', '.', 'And', 'a', 'little', 'racist', '.']
[7, 4, 5] ['that'] notMention []

[7, 10, 12] ['a', 'little'] notMention []

--
['Sheldon', '

In [11]:
data = read_turkle_annotation_multiple_scene('data/institute_name_pilot/pilot_1.csv')
golden = read_turkle_annotation_multiple_scene('data/pilot_2/golden.csv')
for item in golden:
    item['WorkerId'] = "golden"

results = []
results.extend(data)
results.extend(golden)
scenes = gather_by_scene(results)

In [13]:
keys = ['sentenceIndex', 'startToken', 'endToken']

for scene_key in scenes:
    print(scene_key)
    scene = scenes[scene_key]
    sentences = scene[0]['sentences']
    for sent in sentences:
        print(sent)
    spans = scene[0]['answer_spans']
    # Query
    for i in range(len(spans)):
        current_span = spans[i]
        query = [current_span['querySpan'][x] for x in keys]
        query_text = sentences[query[0]][query[1]: query[2]]
        # print(current_span)
        # if query_text[0]!="that":
        #     continue
        print(query, query_text)

        for k in range(len(scene)):
            annotator = scene[k]['WorkerId']
            current_span = scene[k]['answer_spans'][i]
            answers = []
            if (not current_span['notPresent']) and (not current_span['notMention']):
                for item in current_span['span_list']:
                    temp = [item[x] for x in keys]
                    temp.append(current_span['notSure'])
                    answers.append(temp)

            answer_texts = []
            for item in answers:
                answer_texts.append(sentences[item[0]][item[1]: item[2]])
            if answers==[]:
                if current_span['notPresent']:
                    print("notPresent", [x[-1] for x in answers], annotator)
                elif current_span['notMention']:
                    print("notMention", [x[-1] for x in answers], annotator)
            else:
                print(answers, answer_texts, [x[-1] for x in answers], annotator)
        print()

    # Annotations and Annotator Name

    # anno1 = scene[0]
    # anno2 = scene[1]
    # anno3 = scene[2]
    # anno4 = scene[3]
    #
    # for a, b, c, d in zip(anno1['answer_spans'], anno2['answer_spans'], anno3['answer_spans'], anno4['answer_spans']):
    #     print(a)
    #     print(b)
    #     print(c)
    #     print(d)
    #     print()
    print("=="*50)

:|Where|'d|you|go|?|I|ca|n't
['Penny', ':', 'Where', "'d", 'you', 'go', '?', 'I', 'ca', "n't", 'tell', 'if', 'the', 'turkey', "'s", 'done', '!']
['Leonard', ':', 'Be', 'right', 'there', '!', 'Hi', ',', 'lover', '.']
['Penny', ':', 'What', 'are', 'you', 'doing', '?']
['Leonard', ':', 'I', "'", 'm', 'sorry', 'about', 'the', 'journal', '.', 'I', 'want', 'to', 'make', 'it', 'up', 'to', 'you', '.', 'So', 'I', "'", 'm', 'gon', 'na', 'let', 'you', 'post', 'a', 'shame', 'photo', 'of', 'me', 'on', 'Facebook', '.']
['Penny', ':', 'I', 'am', 'not', 'putting', 'that', 'on', 'the', 'Internet', '!', 'I', 'do', "n't", 'want', 'people', 'to', 'see', 'this', '.', 'I', 'do', "n't", 'want', 'to', 'see', 'it', '!']
['Leonard', ':', 'Do', "n't", 'want', 'people', 'to', 'see', 'what', ',', 'huh', '?', 'A', 'little', 'bit', 'of', 'this', '?']
['Penny', ':', 'Oh', '.']
['Leonard', ':', 'Some', 'of', 'this', '?', 'And', ',', 'since', 'it', "'s", 'Thanksgiving', ',', 'an', 'extra', 'helping', 'of', 'this', '?']

In [ ]:
# Last Scene: The first token : We Plural
# X squared time, it
#